# Heuristic Approach for Segment-based Tiering Decisions

In [16]:
import pandas as pd
import csv 
import os

from operator import itemgetter

In [17]:
# performance measurements of the benchmark queries
df_perf = pd.read_csv('../data/benchmark/runtimes.csv')
# memory consumption of segments
df_memory = pd.read_csv('../data/benchmark/memory_consumption.csv')
# memory consumption of indexes
df_memory_index = pd.read_csv('../data/benchmark/memory_consumption_index.csv')

# calibration data for hyrise 
df_poslist_scan_penalty = pd.read_csv('../data/calibration/poslist_scan_penalty.csv')
# calibration data for storage devices
df_storage_penalty = pd.read_csv('../data/calibration/storage_penalty.csv')

# workload definition 
df_workload = pd.read_csv('../data/workloads/workload_1/workload.csv')
# chunk access statistics 
df_chunk_access = pd.read_csv('../data/workloads/workload_1/chunk_access.csv')

# Apply indexes on pandas dataframes
df_perf.set_index(['ORDER_BY', 'ENCODING', 'SCAN_COLUMN', 'SELECTIVITY', 'INDEX', 'SCAN_TYPE'], inplace=True)
df_memory.set_index(['ORDER_BY', 'ENCODING', 'COLUMN', 'CHUNK_ID'], inplace=True)
df_memory_index.set_index(['ORDER_BY', 'ENCODING', 'COLUMN', 'CHUNK_ID'], inplace=True)
df_poslist_scan_penalty.set_index(['ENCODING'], inplace=True)
df_storage_penalty.set_index(['STORAGE', 'ENCODING', 'INDEX'], inplace=True)
df_chunk_access.set_index(['QUERY_ID', 'CHUNK'], inplace=True)

In [18]:
E = df_perf.index.levels[1].unique()           # encoding 
N = df_perf.index.levels[2].unique()           # scan column 
M = df_memory.index.levels[3].unique()         # chunks
O = df_perf.index.levels[0].unique()           # order_by
I = [0,1]                                      # index
D = [0,1]                                      # storage device

### Methods

In [19]:
# copied from data_transformation file
DICT_ENCODINGS = ["Dictionary", "Unencoded", "LZ4", "RunLength", "FoR-SIMD"]
DICT_COLUMNS = ["captain_id", "latitude", "longitude", "timestamp", "captain_status"]

def display_conf_values(segment):
    s  = '- ' if segment[3] != segment[1] else 'S '
    s += '- ' if segment[4] == 0 else 'I '
    s += DICT_ENCODINGS[segment[2]]
    return s

def print_result(config, b):
    print('Solving for budget of ', b)
    print('Objective: ', objective(config))
    for d in D:
        print('Strorage Consumption', d, ':', budget(config, d))
    print('')
    
    longest_str = max([len(encoding) for encoding in DICT_ENCODINGS]) + 10
    
    for d in D:
        print('Storage:', d)
    
        # print header
        for column_name in ['CHUNK'] + DICT_COLUMNS:
            print(column_name.ljust(longest_str), end='')
        print('')
    
        for m in M:
            print(str(m).ljust(longest_str), end='')
            for n in N:
                segment = config[(m,n)]
                # check if current storage is selected 
                if segment[5] == d:
                    print(display_conf_values(segment).ljust(longest_str), end='')
                else:
                    print(' '.ljust(longest_str), end='')
            print('') 
        print('\n')

### Export

In [20]:
# settings for csv export of configurations
FOLDER = '../../data/config/'
STATS_FILE = '../../data/config_benchmark/model_stats.csv' 
MODEL_NAME = 'GH_SEGMENT'
CONVERSION_FACTOR = 1_000_000 # Bytes to Megabytes 

def get_file_name(b, alpha):
    return FOLDER + MODEL_NAME + str(alpha).replace('.','-') + '_' + str(b//CONVERSION_FACTOR) + '-' + str(ssd_budget//CONVERSION_FACTOR) + '.csv'

def budget_string(budget1, budget2):
    return str(budget1/CONVERSION_FACTOR) + '|' + str(budget2/CONVERSION_FACTOR)

def write_file(file_path, header, lines):          
    with open(file_path, mode='w') as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(header)
           
        for line in lines:
            csv_writer.writerow(line)
                
    out_file.close()
    
def export_config(config, b, alpha):
    lines = []
    
    for m in M:
        for n in N:
            lines += [config[(m,n)][:6]]
            
    write_file(get_file_name(b, alpha), ['CHUNK', 'COLUMN', 'ENCODING', 'SORT', 'INDEX', 'STORAGE'], lines)
    
def export_stats(config, alpha, b, ssd_budget):
    create_file = True
    if os.path.isfile(STATS_FILE):
        create_file = False
    with open(STATS_FILE, 'a') as out_file:
        csv_writer = csv.writer(out_file)
        if create_file:
            csv_writer.writerow(['MODEL', 'STORAGE_UNITS', 'BUDGET', 'OBJECTIVE', 'MEMORY', 'SOLVER_TIME'])
        line = [MODEL_NAME + ' (' + str(alpha) + ')', '2', budget_string(b, ssd_budget) , objective(config), budget_string(budget(config, 0), budget(config, 1)), '']
        csv_writer.writerow(line)
    out_file.close()

In [21]:
def value_func(memory, performance, alpha):
    if performance == 0:
        return 0
    return 1 / (memory * (performance**alpha))

def scan_order_penalty(scan_order, encoding):
    if scan_order == 0:
        return 1
    penalty = {
        0:2.1, #1.1, # 1.1 big data
        1:2.1, # 3.1 big data
        2:4, # 7.2 big data
        3:3.7, # 6.1 big data
        4:1.6 # 2.7 big data
    } 
    return penalty.get(encoding, "Encoding id is not defined")

def storage_penalty(e, i, b):
    if b == 0:
        return 1
    elif i == 1:
        #penalty = {0:3, 1:1, 2:1, 3:1, 4:1}
        penalty = {0:32, 1:1, 2:1, 3:1, 4:1} 
        return penalty.get(e, "Encoding id is not defined")
    else:
        #["Dictionary", "Unencoded", "LZ4", "RunLength", "FoR-SIMD"]
        #penalty = {0:19, 1:28, 2:10, 3:22, 4:8} --> median
        penalty = {0:55, 1:46, 2:10, 3:45, 4:20} 
        return penalty.get(e, "Encoding id is not defined")

def segment_accessed(m, q):
    if df_chunk_access.loc[(q, m)]['ACCESSED'] == 0:
        return 0
    return (1/df_chunk_access.loc[(q, slice(None))]['ACCESSED'].sum())

def scan_performance(m, n, e, o, i, d, scan):    
    # set index value to 0 for all scans with a scan order value >= 1
    # indexes can only be used to speed up the first scan
    index_value = i if scan['SCAN_ORDER'] == 0 else 0
    
    return df_perf.loc[(o, e, scan['SCAN_COLUMN'], scan['SELECTIVITY'], index_value, scan['SCAN_TYPE'])]['TIME'] * \
           segment_accessed(m, scan['QUERY_ID']) * scan['FREQUENCY'] * scan['SCAN_FACTOR'] * \
           scan_order_penalty(scan['SCAN_ORDER'], e) * storage_penalty(e, index_value, d)
    
def aggregated_segment_runtime(m, n, e, o, i, d):
    runtime = 0 
    for scan_id in range(len(df_workload)):
        scan = df_workload.iloc[scan_id]
        if scan['SCAN_COLUMN'] == n:
            runtime += scan_performance(m, n, e, o, i, d, scan)
    return runtime

def memory_footprint(m, n, e, o, i):    
    if i == 0:
        return df_memory.loc[(o,e,n,m)]['SIZE_IN_BYTES']
    else:
        return df_memory.loc[(o,e,n,m)]['SIZE_IN_BYTES'] + df_memory_index.loc[(o,e,n,m)]['SIZE_IN_BYTES']

### Config Methods

In [22]:
def budget(config, d):
    b = 0
    for key in config:
        if  config[key][5] == d:
            b += config[key][6]
    return b

def objective(config):
    obj = 0
    for key in config:
        s = config[key] 
        obj += aggregated_segment_runtime(s[0], s[1], s[2], s[3], s[4], s[5])
    return obj

def update_config_segment(config, segment):
    config[tuple(segment[:2])] = segment
    return config

def update_option_list(option_list, benefits):
    new_list = []
    for i in benefits:
        if i[2] > 0:
            new_list += [option_list[i[0]]]
    return new_list

# Create dict with all possible options  
def calc_options(chunks, columns, encodings, indexes, orderings, devices, alpha):
    options = {}
    for chunk in chunks:
        options[chunk] = []
        for column in columns:
            for encoding in encodings:
                for ordering in orderings:
                    for index in indexes:
                        for device in devices:
                            memory = memory_footprint(chunk, column, encoding, ordering, index)
                            value = value_func(memory, aggregated_segment_runtime(chunk, column, encoding, ordering, index, device), alpha)
                            options[chunk] += [[chunk, column, encoding, ordering, index, device, memory, value]]
    return options

# determine initial configuration 
# select for each chunk the segment with the highest value function
# fill up all other segments with the lowest memory consumption 
def calc_init_config(options):
    init_config = {}
    for chunk in options:
        # sort options list for a chunk by value function value 
        # sort by memory 6 --> be able to calculate small configurations
        # sort by value 7 
        sorted_list = sorted(options[chunk], key=itemgetter(6), reverse=True)
        
        # set segment with highest value that defines the ordering of the entire chunk 
        chunk_ordering = sorted_list[0][3]
        
        # set all other segments to the one with the lowest memory consumption
        for i in range(len(options[chunk])):
            segment = options[chunk][i]
            # ignore all segment configs with different ordering
            # ignore all segment configs in DRAM 
            if chunk_ordering == segment[3] and segment[5] == 1:
                if tuple(segment[:2]) not in init_config.keys():
                    init_config = update_config_segment(init_config, segment)
                else:
                    if segment[6] < init_config[tuple(segment[:2])][6]:
                        init_config = update_config_segment(init_config, segment) 

    return init_config

# remove all config options with different ordering settings
def create_option_list(options, config):
    option_list = []
    for chunk in options:
        ordering = config[(chunk, 0)][3]
        for segment in options[chunk]:
            if segment[3] == ordering:
                option_list += [segment]
    return option_list
                        
def setup(alpha):
    options = calc_options(M, N, E, I, O, D, alpha)
    init_config = calc_init_config(options)
    option_list = create_option_list(options, init_config)
    
    print('Initial Configuration Generated')
    for d in D:
        print('Budget', d, ':', budget(init_config, d))
    print('Options: ', len(option_list), '\n')
    
    return init_config, option_list 

# returns a sorted list of segments with the memory and performance diff 
# compared to the current segment in the given config (sorted by performance)
def calc_benefits(options, config, d):
    benefits = []
    for i in range(len(options)):
        segment = options[i]
        config_segment = config[tuple(segment[:2])]
        if segment[5] == d:
            if config_segment == d:
                memory = segment[6] - config_segment[6]
            else:
                memory = segment[6]
            
            benefits += [[i, memory, segment[7] - config_segment[7]]]
    return sorted(benefits, key=itemgetter(2), reverse=True)    

def greedy_optimization(config, option_list, budgets):
    for d in D:
        changed_config = True
        b = budgets[d] - budget(config, d)
        print(b)
        
        while (changed_config):
            changed_config = False
            benefits = calc_benefits(option_list, config, d)
            
            # select element with highest performance gain that fits into the memory budget
            for i in benefits:
                if (i[2] > 0) and (i[1] <= b):
                    b -= i[1]
                    config = update_config_segment(config, option_list[i[0]])
                    changed_config = True
                    break 
    
    return config, update_option_list(option_list, benefits)

In [23]:
alpha = 2
ssd_budget = 3_000_000_000
config_init, option_list_init = setup(alpha)

for b in range( 0, 11_000_000_000, 1_000_000_000):
    config = config_init
    option_list = option_list_init
    if budget(config, 1) > ssd_budget:
        print('No solution for given budget: ', b, '\n')
    else:
        greedy_optimization(config, option_list, [b, ssd_budget])
        print_result(config, b)
        #export_config(config, b, alpha)

Initial Configuration Generated
Budget 0 : 0
Budget 1 : 2842123322
Options:  1000 

0
157876678
Solving for budget of  0
Objective:  23895705702774.984
Strorage Consumption 0 : 0
Strorage Consumption 1 : 2855936618

Storage: 0
CHUNK               captain_id          latitude            longitude           timestamp           captain_status      
0                                                                                                                       
1                                                                                                                       
2                                                                                                                       
3                                                                                                                       
4                                                                                                                       
5                                              